In [1]:
import scipy


Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

In [2]:
# обозначим данные 
mean_predict = 9.5
mean_observed = 9.57
n_samples = 160
std = 0.4

In [8]:
import numpy as np
from scipy import stats
def count_level(mean, std, n_size, mean_observed):
    std_error_cnt = std / np.sqrt(n_size)
    z_point = (mean_observed - mean) / std_error_cnt
    t_distribution = stats.norm.cdf(z_point)
    return 2 * (1 - t_distribution)

In [9]:
print('%.4f' % count_level(mean_predict, std, n_samples, mean_observed))

0.0269


Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

* линейную регрессию с помощью LinearRegression без параметров

* случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. 
Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости α=0.05?

In [12]:
# Импортируем необходимые модули
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [24]:
# импортируем данные из diamonds.txt
data = pd.read_csv('diamonds.txt', sep = '\t')

In [25]:
# проверим корректность импорта данных
data.head(5)

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [26]:
# отделим метку класса, а именно цену
price = data.price
data.drop('price', axis=1,inplace=True)

In [27]:
print(data)

       carat  depth  table     x     y     z
0       0.23   61.5   55.0  3.95  3.98  2.43
1       0.21   59.8   61.0  3.89  3.84  2.31
2       0.23   56.9   65.0  4.05  4.07  2.31
3       0.29   62.4   58.0  4.20  4.23  2.63
4       0.31   63.3   58.0  4.34  4.35  2.75
...      ...    ...    ...   ...   ...   ...
53935   0.72   60.8   57.0  5.75  5.76  3.50
53936   0.72   63.1   55.0  5.69  5.75  3.61
53937   0.70   62.8   60.0  5.66  5.68  3.56
53938   0.86   61.0   58.0  6.15  6.12  3.74
53939   0.75   62.2   55.0  5.83  5.87  3.64

[53940 rows x 6 columns]


In [28]:
# Разделим данные на обучение и тест с помощь train_test_split
train_data, test_data, price_train, price_test = train_test_split(data, price, test_size = 0.25, random_state = 1)

In [32]:
# Обучим два классификатора - линейный и случайный лес
model_linear = LinearRegression()
model_forest = RandomForestRegressor(n_estimators=10, random_state=1)

model_linear.fit(train_data, price_train)
model_forest.fit(train_data, price_train)

RandomForestRegressor(n_estimators=10, random_state=1)

In [33]:
# предскажем значения с помощью двух моделей
predicted_linear = model_linear.predict(test_data)
predicted_forest = model_forest.predict(test_data)

In [38]:
linear_diff = abs(predicted_linear - price_test)
forest_diff = abs(predicted_forest - price_test)

In [40]:
#print(linear_diff)

In [41]:
# Проведем t_test
stats.ttest_rel(linear_diff, forest_diff)

Ttest_relResult(statistic=13.017729783878252, pvalue=1.6551745751479057e-38)

В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).



In [44]:
from statsmodels.stats.weightstats import DescrStatsW

In [45]:
stats_data = DescrStatsW(linear_diff - forest_diff)

In [49]:
print(stats_data.tconfint_mean())

(74.28724532595251, 100.62452098634108)
